In [1]:
import pandas as pd
import numpy as np
import sqlite3

In [3]:
con = sqlite3.connect('db')

In [6]:
!gdown --id 1Vzzp3iopwUHnznj0Zf0DVu6uUwwqxtEL

/usr/local/lib/python3.10/dist-packages/gdown/cli.py:121: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=1Vzzp3iopwUHnznj0Zf0DVu6uUwwqxtEL
To: /content/german_credit_augmented.csv
100% 69.6k/69.6k [00:00<00:00, 3.34MB/s]


In [7]:
df = pd.read_csv('/content/german_credit_augmented.csv')

In [8]:
df.head()

,age,sex,job,housing,saving_accounts,checking_account,credit_amount,duration,purpose,default,contract_dt,client_id
0,33,male,2,own,NaN,NaN,3074,9,radio/TV,0,2008-06-29 18:52:00,210
1,43,male,1,own,little,little,1344,12,car,0,2007-05-20 18:30:19,929
2,52,male,2,own,quite rich,NaN,936,9,education,0,2008-04-27 08:23:07,200
3,35,female,3,own,little,NaN,1393,11,car,0,2007-05-06 10:58:22,45
4,28,male,2,own,little,NaN,776,12,radio/TV,0,2007-07-21 13:22:14,358


In [9]:
df['contract_dt'] = pd.to_datetime(df['contract_dt'],format='%Y-%m-%d %H:%M:%S')

In [10]:
df.to_sql('german_credit',con,index=False,if_exists='replace')

1000

In [11]:
def select(sql):
  return pd.read_sql(sql,con)

In [20]:
sql = '''select *, t.age * 3 as age_mult3, t.housing from german_credit as t
limit 10'''
select(sql)

,age,sex,job,housing,saving_accounts,checking_account,credit_amount,duration,purpose,default,contract_dt,client_id,age_mult3,housing
0,33,male,2,own,None,None,3074,9,radio/TV,0,2008-06-29 18:52:00,210,99,own
1,43,male,1,own,little,little,1344,12,car,0,2007-05-20 18:30:19,929,129,own
2,52,male,2,own,quite rich,None,936,9,education,0,2008-04-27 08:23:07,200,156,own
3,35,female,3,own,little,None,1393,11,car,0,2007-05-06 10:58:22,45,105,own
4,28,male,2,own,little,None,776,12,radio/TV,0,2007-07-21 13:22:14,358,84,own
5,27,female,2,own,little,moderate,1295,18,furniture/equipment,0,2008-06-18 04:10:05,86,81,own
6,29,female,2,own,little,moderate,959,9,furniture/equipment,1,2007-05-21 14:32:00,351,87,own
7,42,female,2,rent,rich,rich,409,12,radio/TV,0,2008-06-05 05:31:13,27,126,rent
8,26,male,2,own,little,little,4370,42,radio/TV,1,2007-11-29 00:20:44,639,78,own
9,55,male,2,free,rich,None,1555,12,repairs,1,2007-09-11 20:27:28,754,165,free


In [25]:
sql = '''select count(*) from german_credit t
where t.contract_dt between '2007-01-01' and '2007-12-31'
'''
select(sql)

,count(*)
0,573


In [26]:
sql = '''select * from german_credit t
where t.contract_dt between '2007-01-01' and '2007-12-31'
and t.purpose in ('car' ,'repairs')

order by t.contract_dt desc, credit_amount
'''
select(sql)

,age,sex,job,housing,saving_accounts,checking_account,credit_amount,duration,purpose,default,contract_dt,client_id
0,36,male,3,rent,None,moderate,7057,20,car,0,2007-12-29 16:10:08,99
1,30,male,2,own,little,moderate,639,12,repairs,1,2007-12-28 17:09:43,127
2,25,male,2,rent,moderate,moderate,1264,15,car,1,2007-12-28 08:38:58,979
3,48,male,2,own,little,None,2134,9,car,0,2007-12-24 16:28:30,20
4,67,female,2,own,little,moderate,3872,18,repairs,0,2007-12-24 12:25:28,779
...,...,...,...,...,...,...,...,...,...,...,...,...
192,55,male,2,own,rich,None,1413,12,car,0,2007-05-05 06:35:11,209
193,55,female,0,free,little,little,1190,18,repairs,1,2007-05-05 00:14:17,429
194,47,male,3,own,little,moderate,1209,6,car,1,2007-05-03 10:29:01,485
195,36,male,2,own,little,moderate,884,18,car,1,2007-05-02 06:22:11,184


In [28]:
t = pd.DataFrame({
              'purpose':['car','for a car','to buy a car','automobile','else'],
              'amount':[1000,400,600,700,1500]})

In [29]:
t.to_sql('purpose',con,index=False,if_exists='replace')

5

In [30]:
sql = '''select * from purpose t'''
select(sql)

,purpose,amount
0,car,1000
1,for a car,400
2,to buy a car,600
3,automobile,700
4,else,1500


In [31]:
sql = '''select * from purpose t
where (t.purpose like '%car%'
or t.purpose like '%auto%') and t.amount > 500
'''
select(sql)

,purpose,amount
0,car,1000
1,to buy a car,600
2,automobile,700


In [32]:
sql = '''select
avg(case when t.credit_amount > 1000 then 1 else 0 end) as greater_1000_frac
from german_credit t'''
select(sql)

,greater_1000_frac
0,0.884


In [33]:
cur = con.cursor()

In [34]:
sql = '''
drop table if exists greater_1000_credit;
create table greater_1000_credit as
select * from german_credit t
where t.credit_amount > 1000'''

In [35]:
cur.executescript(sql)

In [36]:
sql = '''select * from greater_1000_credit t'''
select(sql)

,age,sex,job,housing,saving_accounts,checking_account,credit_amount,duration,purpose,default,contract_dt,client_id
0,33,male,2,own,None,None,3074,9,radio/TV,0,2008-06-29 18:52:00,210
1,43,male,1,own,little,little,1344,12,car,0,2007-05-20 18:30:19,929
2,35,female,3,own,little,None,1393,11,car,0,2007-05-06 10:58:22,45
3,27,female,2,own,little,moderate,1295,18,furniture/equipment,0,2008-06-18 04:10:05,86
4,26,male,2,own,little,little,4370,42,radio/TV,1,2007-11-29 00:20:44,639
...,...,...,...,...,...,...,...,...,...,...,...,...
879,65,male,2,free,little,little,2600,18,radio/TV,1,2007-12-16 20:17:19,624
880,30,male,3,own,little,moderate,4455,36,business,1,2007-07-12 14:08:58,181
881,33,male,2,own,little,moderate,6403,24,radio/TV,0,2008-04-08 03:24:26,730
882,29,female,2,own,None,None,5003,21,car,1,2007-11-29 15:51:45,557


In [37]:
jan = pd.DataFrame({'month':['jan','jan'],'revenue':[1,2]})
feb = pd.DataFrame({'month':['feb','feb'],'revenue':[1,2]})

In [38]:
jan.to_sql('jan',con,index=False,if_exists='replace')
feb.to_sql('feb',con,index=False,if_exists='replace')

2

In [40]:
sql = '''select t.revenue,
'jan' as month
from jan t
union all
select t.revenue,
'feb' as month
from feb t'''
select(sql)

,revenue,month
0,1,jan
1,2,jan
2,1,feb
3,2,feb
